<a href="https://colab.research.google.com/github/Arul-Vincy-Mary/nerd/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#data visualization
!pip install pyLDAvis

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [7]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [8]:
#reading the data
import pandas as pd
import numpy as np
import email

In [9]:
data_1 = pd.read_csv("/content/drive/MyDrive/emails.csv")

In [10]:
#copy of the dataset
data = data_1.copy(deep=True)

In [11]:
data.head(5)

,file,message
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...


In [12]:
data.shape

(517401, 2)

In [13]:
#message
print(data.loc[1]['message'])

Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>
Date: Fri, 4 May 2001 13:51:00 -0700 (PDT)
From: phillip.allen@enron.com
To: john.lavorato@enron.com
Subject: Re:
Mime-Version: 1.0
Content-Type: text/plain; charset=us-ascii
Content-Transfer-Encoding: 7bit
X-From: Phillip K Allen
X-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>
X-cc: 
X-bcc: 
X-Folder: \Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail
X-Origin: Allen-P
X-FileName: pallen (Non-Privileged).pst

Traveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.

As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not.  Too often the

Extract headers:

In [14]:
def get_field(field, messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get(field))
    return column  

In [15]:
data['subject'] = get_field("Subject", data['message'])
data['X-Folder'] = get_field("X-Folder", data['message'])
data['X-From'] = get_field("X-From", data['message'])
data['X-To'] = get_field("X-To", data['message'])
data.head()   

,file,message,subject,X-Folder,X-From,X-To
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Randall L Gay
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Greg Piper


In [16]:
#extract body of the message
def body(messages):
    column = []
    for message in messages:
        e = email.message_from_string(message)
        column.append(e.get_payload())
    return column

In [17]:
data['body'] = body(data['message'])
data.head()

,file,message,subject,X-Folder,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,"\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Se...",Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,\Phillip_Allen_Dec2000\Notes Folders\'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


Look into X-Folder:

In [18]:
print(data['X-Folder'][0])

# we only want last folder name
data['X-Folder'][0].split("\\")[-1]

\Phillip_Allen_Jan2002_1\Allen, Phillip K.\'Sent Mail


"'Sent Mail"

In [19]:
def preprocess_folder(folders):
    column = []
    for folder in folders:
        if (folder is None or folder == ""):
            column.append(np.nan)
        else:
            column.append(folder.split("\\")[-1].lower())
    return column

data['X-Folder'] = preprocess_folder(data['X-Folder'])
data.head()

,file,message,subject,X-Folder,X-From,X-To,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,,'sent mail,Phillip K Allen,Tim Belden <Tim Belden/Enron@EnronXGate>,Here is our forecast\n\n
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Re:,'sent mail,Phillip K Allen,John J Lavorato <John J Lavorato/ENRON@enronXg...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,Re: test,'sent mail,Phillip K Allen,Leah Van Arsdall,test successful. way to go!!!
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,,'sent mail,Phillip K Allen,Randall L Gay,"Randy,\n\n Can you send me a schedule of the s..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Re: Hello,'sent mail,Phillip K Allen,Greg Piper,Let's shoot for Tuesday at 11:45.


In [20]:
# view some unique folders
data['X-Folder'].unique()[0:40]

array(["'sent mail", 'all documents', 'contacts', 'deleted items',
       'discussion threads', 'inbox', 'notes inbox', 'sent items', 'sent',
       'straw', '2000 conference', 'active international', 'avaya', 'bmc',
       'bridge', 'bristol babcock', 'colleen koenig', 'compaq',
       'computer associates', 'continental airlines', 'cooper cameron',
       'corestaff', 'dell', 'ebs', 'ees', 'enron europe', 'etol', 'fedex',
       'ge', 'hp', 'human resources', "kinko's", 'nepco europe', 'nepco',
       'oec', 'pcc values', 'personal', 'purchasing', 'requisite', 'sap'],
      dtype=object)

Drop the following columns: file,message,date,subject,X-From,X-To

In [21]:
cols_to_drop = ['file','message','subject','X-From','X-To']
data.drop(cols_to_drop, axis=1, inplace=True)
data.head()
data.shape

(517401, 2)

In [22]:
#subset of the data set
data = data[data["X-Folder"]=="'sent mail"]
data.columns
data.shape

(30237, 2)

In [23]:
data.head()

,X-Folder,body
0,'sent mail,Here is our forecast\n\n
1,'sent mail,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.


In [24]:
df = data[:10000]
df.head()

,X-Folder,body
0,'sent mail,Here is our forecast\n\n
1,'sent mail,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.


In [25]:
df_1 = df.copy(deep=True)

Data cleaning:

In [ ]:
#view the emails
for i in range(25):
   print("################################################ EMAIL CONTENT NUMBER:",i,"############################################################################")
   print(df_1.body.iloc[i])

From the above emails, there are many emails contain ----forwarded by--- in it. Removing those for better model

In [27]:
#checking the emails contain ----forwarded by----
df_1.body.str.contains('[- ]*Forwarded by').value_counts()

False    6402
True     3598
Name: body, dtype: int64

Looking at the forwarded texts:

In [28]:
import re
#removing forwarded patterns
def deal_forwarded(row):
    condition = '[- ]*Forwarded by[\S\s]*Subject:[\S\t ]*'                      
    return re.sub(condition, '', row).strip()
df_1['content1'] = df_1.body.map(deal_forwarded)    

In [29]:
print(df_1.content1.str.contains('[- ]*Forwarded by').value_counts())

False    9966
True       34
Name: content1, dtype: int64


There is another pattern of forwarded to clean.

In [ ]:
#checking for forwarded texts
for email in df_1.content1[df_1.content1.str.contains('[- ]*Forwarded by')]:
    print(email)
    print("############################ END OF EMAIL ################################################################")

there is one more pattern:

In [31]:
def deal_forwarded_patternless(row):
    condition = '[- ]*Forwarded by[\S\s]*[-]+'
    return re.sub(condition, '', row).strip()
df_1['content2'] = df_1.content1.map(deal_forwarded_patternless)    

In [32]:
print(df_1.content2.str.contains('[- ]Forwarded by').value_counts())

False    10000
Name: content2, dtype: int64


now there is no forwards in the data.

In [33]:
df_1.head()

,X-Folder,body,content1,content2
0,'sent mail,Here is our forecast\n\n,Here is our forecast,Here is our forecast
1,'sent mail,Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...,Traveling to have a business meeting takes the...
2,'sent mail,test successful. way to go!!!,test successful. way to go!!!,test successful. way to go!!!
3,'sent mail,"Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s...","Randy,\n\n Can you send me a schedule of the s..."
4,'sent mail,Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.,Let's shoot for Tuesday at 11:45.


In [34]:
#Getting the final content2 out:
emails = pd.DataFrame(df_1['content2'])

In [35]:
#Removing the emails that are empty:
emails = emails.loc[~(emails.content2=='')]
emails = emails.rename(columns={'content2':'body'}).reset_index(drop=True)

In [36]:
print(emails.shape)
emails.head()

(9922, 1)


,body
0,Here is our forecast
1,Traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"Randy,\n\n Can you send me a schedule of the s..."
4,Let's shoot for Tuesday at 11:45.


In [37]:
emails = emails.drop_duplicates(subset='body', keep='first',inplace=False)
emails.shape

(9631, 1)

In [39]:
#converting uppercase into lower case
emails['body'] = emails['body'].str.lower()
emails.head()

,body
0,here is our forecast
1,traveling to have a business meeting takes the...
2,test successful. way to go!!!
3,"randy,\n\n can you send me a schedule of the s..."
4,let's shoot for tuesday at 11:45.


In [40]:
#remove contracted
def change_contracted_words(body):
  column = []
  Apos_dict={"'s":" is","n't":" not","'m":" am","'ll":" will",
             "'d":" would","'ve":" have","'re":" are"}
  
  for file in emails['body']:
    for key,value in Apos_dict.items():
      if key in file:
        file = file.replace(key,value)
    column.append(file)
  
  return column

In [41]:
emails['body'] = change_contracted_words(emails['body'])

In [42]:
#removing repeated words
def remove_duplicates(row):
    condition = '[a-zA-Z]+(@|\.)[a-zA-Z.]+'
    return re.sub(condition, '', str(row)).strip()

In [43]:
emails['body'] = emails.body.map(remove_duplicates)

In [ ]:
! pip install texthero

In [45]:
import texthero as hero

In [46]:
emails['body'] = hero.remove_urls(emails['body'])

In [47]:
emails['body'] = hero.remove_html_tags(emails['body'])

In [48]:
emails['body'] = hero.remove_digits(emails['body'])

In [49]:
emails['body'] = hero.remove_punctuation(emails['body'])

In [50]:
emails['body'] = hero.remove_brackets(emails['body'])

In [51]:
emails['body'] = hero.remove_diacritics(emails['body'])

In [52]:
emails['body'] = hero.remove_whitespace(emails['body'])

In [ ]:
emails['body'].isnull().sum()

In [54]:
emails.shape

(9631, 1)

Model building

In [55]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

In [56]:
#Tokenization
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
# NLTK Stop words
nltk.download('stopwords')
from nltk.corpus import stopwords
#spacy for lemmatization
import spacy

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [57]:
#tokenization
emails['nltk_tokens'] = emails['body'].map(lambda r: word_tokenize(r))
emails

,body,nltk_tokens
0,here is our forecast,"[here, is, our, forecast]"
1,traveling to have a business meeting takes the...,"[traveling, to, have, a, business, meeting, ta..."
2,test successful way to go,"[test, successful, way, to, go]"
3,randy can you send me a schedule of the salary...,"[randy, can, you, send, me, a, schedule, of, t..."
4,let is shoot for tuesday at,"[let, is, shoot, for, tuesday, at]"
...,...,...
9917,rick unfortunately i had to cancel i had a sic...,"[rick, unfortunately, i, had, to, cancel, i, h..."
9918,hi guys here is a spreadsheet with the list of...,"[hi, guys, here, is, a, spreadsheet, with, the..."
9919,i think everyone got the list make sure we are...,"[i, think, everyone, got, the, list, make, sur..."
9920,troy i am out of the office this morning i wil...,"[troy, i, am, out, of, the, office, this, morn..."


In [58]:
#convert the tokens into list
data_words = emails['nltk_tokens'].values.tolist()

In [59]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

In [60]:
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

In [61]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]
def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [62]:
#Stop words removing
stop_words = stopwords.words('english')
data_stopwords = remove_stopwords(data_words)

In [63]:
#Bigrams
bigram_words = make_bigrams(data_stopwords)

In [64]:
#lemmatization
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ','VERB','ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [65]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en", disable=['parser', 'ner'])

In [66]:
#lemmatization with noun, adjctive, verb, adverb
data_lemmatized = lemmatization(bigram_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [67]:
#Dictionary
id2word = corpora.Dictionary(data_lemmatized)

In [68]:
#Corpus
texts = data_lemmatized

In [69]:
#Document-term Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [81]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100,
                                            update_every=1,
                                            chunksize=100,
                                            passes=10,
                                            alpha='auto',
                                            per_word_topics=True)

In [82]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
doc_lda

[(0,
  '0.054*"thank" + 0.052*"know" + 0.042*"need" + 0.036*"let" + 0.029*"subject" '
  '+ 0.027*"call" + 0.021*"make" + 0.019*"get" + 0.018*"see" + 0.018*"send"'),
 (1,
  '0.053*"schedule" + 0.052*"meeting" + 0.034*"meter" + 0.032*"discuss" + '
  '0.031*"plan" + 0.031*"balance" + 0.024*"meet" + 0.020*"attend" + '
  '0.019*"item" + 0.018*"hold"'),
 (2,
  '0.081*"deal" + 0.050*"gas" + 0.044*"contract" + 0.031*"volume" + '
  '0.026*"month" + 0.025*"capacity" + 0.022*"ce" + 0.020*"price" + '
  '0.019*"rate" + 0.018*"power"'),
 (3,
  '0.093*"change" + 0.038*"transport" + 0.034*"comment" + 0.026*"marketing" + '
  '0.023*"ee" + 0.022*"ena" + 0.021*"agree" + 0.017*"currently" + 0.015*"bid" '
  '+ 0.013*"post"'),
 (4,
  '0.021*"order" + 0.017*"continue" + 0.017*"person" + 0.016*"move" + '
  '0.014*"new" + 0.014*"group" + 0.013*"issue" + 0.012*"area" + '
  '0.011*"deliver" + 0.011*"work"'),
 (5,
  '0.033*"service" + 0.030*"company" + 0.019*"provide" + 0.017*"agreement" + '
  '0.017*"report" + 0

In [73]:
#Perplexity score
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -7.905784506388007


In [74]:
#Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.40661377273123467


Data vizulaisation:

In [72]:
import pyLDAvis.gensim_models
import pyLDAvis

In [ ]:
#Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)

In [74]:
pyLDAvis.display(vis)